In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import codecs

batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 40000  # Number of samples to train on.
# Path to the data txt file on disk.
path = '/home/santanu/Downloads/Machine Translation/fra-eng/fra.txt'

def read_input(path):
    
    input_texts = []
    target_texts = []
    input_words = set()
    target_words = set()
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t ' + target_text + ' \n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        for word in input_text.split(" "):
            if word not in input_words:
                input_words.add(word)
        for word in target_text.split(" "):
            if word not in target_words:
                target_words.add(word)
    
    input_words = sorted(list(input_words))
    #print(input_words)
    target_words = sorted(list(target_words))
    #print(target_words)
    num_encoder_words = len(input_words)
    num_decoder_words = len(target_words)
    max_encoder_seq_length = max([len(txt.split(" ")) for txt in input_texts])
    max_decoder_seq_length = max([len(txt.split(" ")) for txt in target_texts])
    
    print('Number of samples:', len(input_texts))
    print('Number of unique input tokens:', num_encoder_words)
    print('Number of unique output tokens:', num_decoder_words)
    print('Max sequence length for inputs:', max_encoder_seq_length)
    print('Max sequence length for outputs:', max_decoder_seq_length)
    
    input_word_index = dict(
        [(word, i) for i, word in enumerate(input_words)])
#    print(input_word_index) 
    target_word_index = dict(
        [(word, i) for i, word in enumerate(target_words)])
 #   print(target_word_index)
    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_words),
        dtype='float32')
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_words),
        dtype='float32')
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_words),
        dtype='float32')
    
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, word in enumerate(input_text.split(" ")):
            #print(input_text.split(" "))
            encoder_input_data[i, t, input_word_index[word]] = 1.
        for t, word in enumerate(target_text.split(" ")):
            #print(target_text.split(" "))
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_word_index[word]] = 1.
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_word_index[word]] = 1.
    return encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_words,num_decoder_words,max_encoder_seq_length,max_decoder_seq_length,input_word_index,target_word_index,input_texts,target_texts

    # This Model would be used for end to end training of the Encoder-Decoder Neural Translation Machine 
    # During inference we will extract components out of this Model to perfor translation.

def model_():
  # Encoder Input will have the one hot encoded representation of the source input words in each timestep   
    encoder_inp = Input(shape=(None, num_encoder_words),name='encoder_inp')
  # An LSTM has both a hidden state and a cell state at each timestep. We are going to extract the final hidden 
  # state and the final cell state from the LSTM to provide context to the decoder.The cell state is generally not 
  # available as one of the LSTM    outputs unless specified through "return_state=True"
    encoder = LSTM(latent_dim, return_state=True,name='encoder')
    encoder_out, state_h, state_c = encoder(encoder_inp)
    encoder_states = [state_h, state_c]
  # The decoder input is the target input sequence expressed as a sequence of one hot encoded vectors   
    decoder_inp = Input(shape=(None, num_decoder_words))
    # The decoder needs to emit an output target word at every timestep and hence the parameter return_sequences is set to 
    # True. The decoder is trying to predict its own sequence but at a one time step gap. Also to provide context from the 
    # source language for translation the initial hidden and cell state of the decoder has been set with the last 
    # hidden and cell state of the encoder(that processed the source language text) 
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_out, _, _ = decoder_lstm(decoder_inp,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_words, activation='softmax')
    decoder_out = decoder_dense(decoder_out)
    model_enc_dec  = Model([encoder_inp, decoder_inp], decoder_out)
    encoder_model = Model(encoder_inp, encoder_states)
    return model
    





# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models


def decoder_model(model):

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inp = [decoder_state_input_h, decoder_state_input_c]
    decoder_inp = model.layers[3].input[0]
    decoder_lstm = model.layers[3] 
    decoder_out, state_h, state_c = decoder_lstm(
        decoder_inp, initial_state=decoder_states_inp)
    decoder_states = [state_h, state_c]
    decoder_dense = model.layers[4]
    decoder_out = decoder_dense(decoder_out)
    decoder_model = Model(
        [decoder_inp] + decoder_states_inp,
        [decoder_out] + decoder_states)
    return decoder_model




def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_words))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_word_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_word, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_word_index = np.argmax(output_word[0, -1, :])
        sampled_char = reverse_target_word_index[sampled_word_index]
        decoded_sentence = decoded_sentence + ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_words))
        target_seq[0, 0, sampled_word_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


# Run training

encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_words,num_decoder_words,max_encoder_seq_length,max_decoder_seq_length,input_word_index,target_word_index,input_texts,target_texts = read_input(path)

print(np.shape(encoder_input_data))
print(np.shape(decoder_input_data))
print(np.shape(decoder_target_data))

model = model_enc_dec()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          #epochs=epochs,
          epochs=20,
          validation_split=0.2)
# Save model
model.save('eng_2_french.h5')

encoder_input = model.layers[2].input
h_f = model.layers[2].output[1]
c_f = model.layers[2].output[2]
encoder_states = [h_f,c_f]

encoder_model = Model(encoder_input,[h_f,c_f])
decoder_model = decoder_model(model)
reverse_input_word_index = dict(
    (i, word) for word, i in input_word_index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in target_word_index.items())

#decode_sequence(input_seq)


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,encoder_model,decoder_model)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)




('Number of samples:', 40000)
('Number of unique input tokens:', 8658)
('Number of unique output tokens:', 16297)
('Max sequence length for inputs:', 7)
('Max sequence length for outputs:', 16)


In [53]:

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,encoder_model,decoder_model)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
('Input sentence:', u'Go.')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Run!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Run!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Wow!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Fire!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Help!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Jump.')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Stop!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Stop!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Stop!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Wait!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Wait!')
('Decoded sentence:', u" C'est est \n")
-
('Input sentence:', u'Go on.')
('Decoded sentence:', u' \n')
-
('Input sentence:', u'Go on.')
('Decoded sentence:', u' \n')
-
('Input sente

In [49]:


def decoder_model(model):

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inp = [decoder_state_input_h, decoder_state_input_c]
    decoder_inp = model.layers[3].input[0]
    decoder_lstm = model.layers[3] 
    decoder_out, state_h, state_c = decoder_lstm(
        decoder_inp, initial_state=decoder_states_inp)
    decoder_states = [state_h, state_c]
    decoder_dense = model.layers[4]
    decoder_out = decoder_dense(decoder_out)
    decoder_model = Model(
        [decoder_inp] + decoder_states_inp,
        [decoder_out] + decoder_states)
    return decoder_model

decoder_model(model)

In [48]:
model.layers[3].input

[<tf.Tensor 'input_6:0' shape=(?, ?, 16297) dtype=float32>,
 <tf.Tensor 'lstm_9/while/Exit_2:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'lstm_9/while/Exit_3:0' shape=(?, 256) dtype=float32>]

In [44]:
model.layers[3].input

[<tf.Tensor 'input_6:0' shape=(?, ?, 16297) dtype=float32>,
 <tf.Tensor 'lstm_9/while/Exit_2:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'lstm_9/while/Exit_3:0' shape=(?, 256) dtype=float32>]

In [31]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
#data_path = 'fra-eng/fra.txt'
data_path = '/home/santanu/Downloads/Machine Translation/fra-eng/fra.txt'


# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with codecs.open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)




Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.9207 - val_loss: 0.9694

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_26 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_25/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_25/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


-
Input sentence: Go.
-
Input sentence: Run!
-
Input sentence: Run!
-
Input sentence: Wow!
-
Input sentence: Fire!
-
Input sentence: Help!
-
Input sentence: Jump.
-
Input sentence: Stop!
-
Input sentence: Stop!
-
Input sentence: Stop!
-
Input sentence: Wait!
-
Input sentence: Wait!
-
Input sentence: Go on.
-
Input sentence: Go on.
-
Input sentence: Go on.
-
Input sentence: I see.
-
Input sentence: I try.
-
Input sentence: I won!
-
Input sentence: I won!
-
Input sentence: Oh no!
-
Input sentence: Attack!
-
Input sentence: Attack!
-
Input sentence: Cheers!
-
Input sentence: Cheers!
-
Input sentence: Cheers!
-
Input sentence: Cheers!
-
Input sentence: Get up.
-
Input sentence: Go now.
-
Input sentence: Go now.
-
Input sentence: Go now.
-
Input sentence: Got it!
-
Input sentence: Got it!
-
Input sentence: Got it?
-
Input sentence: Got it?
-
Input sentence: Got it?
-
Input sentence: Hop in.
-
Input sentence: Hop in.
-
Input sentence: Hug me.
-
Input sentence: Hug me.
-
Input sentence: I fel

In [32]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: I see.
Decoded sentence: Je les déteste.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: Je suis montée à bord.

-
Input sentence: I won!
Decode

In [14]:
import numpy as np
import gensim
import codecs
path = '/home/santanu/Downloads/Machine Translation/fra-eng/fra.txt'

num_samples = 10000



def read_and_process_file(path):
    input_texts = []
    target_texts = []
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for line in lines[: min(num_samples,len(lines) - 1)]:
        input_text, target_text = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t ' + target_text + ' \n'
        input_texts.append(input_text.split(" "))
        target_texts.append(target_text.split(" "))
    return target_texts

target_texts = read_and_process_file(path)
        

#sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
word2vec_source = gensim.models.Word2Vec(input_texts, min_count=1)
word2vec_target = gensim.models.Word2Vec(target_texts, min_count=1)






In [28]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
import numpy as np
import codecs

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
embedding_size_eng,embedding_size_french = 100,100
path = '/home/santanu/Downloads/Machine Translation/fra-eng/fra.txt'

def read_input(path):
    
    input_texts = []
    target_texts = []
    input_words = set()
    target_words = set()
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t ' + target_text + ' \n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        for word in input_text.split(" "):
            if word not in input_words:
                input_words.add(word)
        for word in target_text.split(" "):
            if word not in target_words:
                target_words.add(word)
    
    input_words = sorted(list(input_words))
    target_words = sorted(list(target_words))
    num_encoder_words = len(input_words)
    num_decoder_words = len(target_words)
    max_encoder_seq_length = max([len(txt.split(" ")) for txt in input_texts])
    max_decoder_seq_length = max([len(txt.split(" ")) for txt in target_texts])
    
    print('Number of samples:', len(input_texts))
    print('Number of unique input tokens:', num_encoder_words)
    print('Number of unique output tokens:', num_decoder_words)
    print('Max sequence length for inputs:', max_encoder_seq_length)
    print('Max sequence length for outputs:', max_decoder_seq_length)
    print(len(input_words),len(target_words))
    
    input_word_index = dict(
        [(word, i) for i, word in enumerate(input_words)])
    target_word_index = dict(
        [(word, i) for i, word in enumerate(target_words)])
    
    print(len(input_word_index.keys()))
    print(len(target_word_index.keys()))
    
    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length),
        dtype='float32')
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length),
        dtype='float32')
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length,1),
        dtype='float32')
    
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, word in enumerate(input_text.split(" ")):
            encoder_input_data[i, t] = input_word_index[word]
        for t, word in enumerate(target_text.split(" ")):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t] = target_word_index[word]
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1,0] = target_word_index[word]
    return encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_words,num_decoder_words,max_encoder_seq_length,max_decoder_seq_length,input_word_index,target_word_index  

encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_words,num_decoder_words,max_encoder_seq_length,max_decoder_seq_length,input_word_index,target_word_index = read_input(path)

english_emb,french_emb = np.zeros((num_encoder_words,100)), np.zeros((num_decoder_words,100))
for i,k in enumerate(input_word_index.keys()):
    english_emb[i] = input_word_index[k]
for i,k in enumerate(target_word_index.keys()):
    french_emb[i] = target_word_index[k]

    

print(np.shape(encoder_input_data))
print(np.shape(decoder_input_data))
print(np.shape(decoder_target_data))
# Define an input sequence and process it.
encoder_inp = Input(shape=(max_encoder_seq_length,))
encoder_inp1 = Embedding(num_encoder_words,100, weights=[english_emb])(encoder_inp)
encoder = LSTM(latent_dim, return_state=True)
encoder_out, state_h, state_c = encoder(encoder_inp1)
# We discard `encoder_out` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inp = Input(shape=(max_decoder_seq_length,))
decoder_inp1 = Embedding(num_decoder_words,100,weights=[french_emb])(decoder_inp)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_out, _, _ = decoder_lstm(decoder_inp1,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_words, activation='softmax')
decoder_out = decoder_dense(decoder_out)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inp, decoder_inp], decoder_out)

# Run training
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          #epochs=epochs,
          epochs=20,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inp, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inp = [decoder_state_input_h, decoder_state_input_c]
decoder_out, state_h, state_c = decoder_lstm(
    decoder_inp, initial_state=decoder_states_inp)
decoder_states = [state_h, state_c]
decoder_out = decoder_dense(decoder_out)
decoder_model = Model(
    [decoder_inp] + decoder_states_inp,
    [decoder_out] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.



def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_words))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_word_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_word, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_word_index = np.argmax(output_word[0, -1, :])
        sampled_char = reverse_target_word_index[sampled_word_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_words))
        target_seq[0, 0, sampled_word_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 3120
Number of unique output tokens: 6020
Max sequence length for inputs: 5
Max sequence length for outputs: 12
3120 6020
3120
6020
(10000, 5)
(10000, 12)
(10000, 12, 1)
Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 5s 646us/step - loss: 3.5581 - val_loss: 2.8879

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_22 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_21/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_21/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


ValueError: Input 0 is incompatible with layer lstm_22: expected ndim=3, found ndim=2

In [23]:
len(input_word_index.keys())
len(target_word_index.keys())
len(input_words)

NameError: name 'input_words' is not defined